In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import cred
import time
import random

scope = "playlist-read-private playlist-modify-public user-read-private user-top-read user-library-read"
auth_manager = SpotifyOAuth(client_id=cred.client_id, client_secret=cred.client_secret, redirect_uri='http://127.0.0.1:8080', scope=scope)
sp = spotipy.Spotify(auth_manager=auth_manager)

user_id = sp.current_user()['id']
user_country = sp.current_user()['country']


In [ ]:
def get_user_playlist_ids():
  '''
  Collects a list of user playlist dictionaries and the Spotify ID for each of them.
  '''
  playlists_lst =[]
  ids = []
  offset = 0
  while True:
      playlists = sp.current_user_playlists(offset=offset)
      if len(playlists['items']) == 0:
          break
      for playlist in playlists['items']:
          playlists_lst.append(playlist)
      offset = offset + len(playlists['items'])
      time.sleep(0.1) 
  
  for playlist in playlists_lst:
      ids.append(playlist['id'])
  return ids, playlists_lst

def get_saved_tracks():
  '''
  Gets a user's "liked" tracks
  '''
  ids = []
  print('I\'m starting to look at the user\'s saved tracks!!')
  offset = 0
  t1 = time.time()
  while True:
      track_ids = sp.current_user_saved_tracks(offset=offset)
      if len(track_ids['items']) == 0:
          break
      for track in track_ids['items']:
          if track['track'] == None:
              continue
          else:
              ids.append(track['track']['id'])
      offset = offset + len(track_ids['items'])
      time.sleep(0.1)
  t2 = time.time()
  print(f'Hmmm... getting the liked tracks took {t2-t1} seconds!\n')
  return list(set(ids))

def get_playlist_names(playlists):
  '''
  Returns a list of a user's playlist titles when given a list of playlist ids
  '''
  names = []
  for playlist in playlists:
      name = playlist['name']
      names.append(name)
  return names

def get_song_ids_from_playlists(user, playlist_urls):
  '''
  Gets song ids from each of the songs in given playlist ids
  '''
  ids = []
  t1 = time.time()
  for i in range(len(playlist_urls)):
      offset = 0
      print(f'I\'m starting playlist number {i+1} out of {len(playlist_urls)}')
      while True:
          track_ids = sp.user_playlist_tracks(user=user, playlist_id=playlist_urls[i], offset=offset, fields ='items.track.id')
          #print(track_ids)
          #print(len(track_ids['items']))
          if len(track_ids['items']) == 0:
              break
          for track in track_ids['items']:
              if track['track'] == None:
                  continue
              else:
                  ids.append(track['track']['id'])
          offset = offset + len(track_ids['items'])
          time.sleep(0.1)
  t2 = time.time()
  print(f'Getting song ids from all those playlists took {round(t2-t1, 2)} seconds!\n')
  return list(set(ids))

def get_recc_ids(list_seed_tracks, country):
  '''
  Gets ids for # recommended songs for each song in the seed tracks list
  '''
  print('Starting to collect recommendation ids.')
  if len(list_seed_tracks) > 150:
      print(f'Wow! I have {len(list_seed_tracks)*30} to make. This may take a while.\n')
  recc_ids = []
  #raw_recs = []
  t1 = time.time()
  for seed in list_seed_tracks:
      seed_to_use = []
      seed_to_use.append(seed)
      recs = sp.recommendations(seed_tracks=seed_to_use, limit = 30, country=country)
      #raw_recs.append(recs)
      #print(recs)
      for i in range(len(recs['tracks'])):
        track_id = recs['tracks'][i]['id']
        recc_ids.append(track_id)
      #print(len(recc_ids))
  set_ids = set(recc_ids) 
  t2 = time.time()
  print(f'Making and saving all of those recommendations took {round(t2-t1, 2)} seconds.\n')
  return list(set_ids)

def create_playlist(tracks):
  sp.user_playlist_create(user_id, 'your recommended songs', description='yay new songs!')
  user_playlists, y = get_user_playlist_ids()
  sp.user_playlist_add_tracks(user_id, user_playlists[0], tracks)
  return 'Your playlist has been created!'

In [ ]:
top_artists = sp.current_user_top_artists(time_range='medium_term', limit=30)
topartists = []
for i, item in enumerate(top_artists['items']):
    topartists.append(item['name'])
    print(i + 1, item['name'])
   
print('\n')
top_tracks = sp.current_user_top_tracks(time_range='medium_term', limit=30)
toptracks = []
for i, item in enumerate(top_tracks['items']):
    toptracks.append(item['name'])
    print(i + 1, item['name'], '//', item['artists'][0]['name'])
    
artist_popularities = []
for i in range(len(top_artists['items'])):
    artist_popularities.append(top_artists['items'][i]['popularity'])
    
track_popularities = []
for i in range(len(top_tracks['items'])):
    track_popularities.append(top_tracks['items'][i]['popularity'])